In [121]:
import yelpapi
import json
import requests
from  pprint import  pprint
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

#reading in county data
counties=pd.read_csv("acs2015_county_data.csv")
counties["county_string"] = counties["County"] + " County, " + counties["State"]
listofcounties = counties["county_string"].tolist()
#reading in zipcodes
zipcodes=pd.read_csv("zip_codes_states.csv")
zipcodes["county_string"] = zipcodes["county"] + " County, " + zipcodes["state"]
listofzipcodes=zipcodes["county_string"].unique().tolist()
counties.to_csv("counties.csv")
zipcodes.to_csv("zcs.csv")
fullcountylist = list(set(listofcounties) & set(listofzipcodes))
counties.head(1)

,CensusId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,...,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,county_string
0,1001,Alabama,Autauga,55221,26745,28476,2.6,75.8,18.5,0.4,...,1.3,1.8,26.5,23986,73.6,20.9,5.5,0.0,7.6,"Autauga County, Alabama"


In [122]:
#creating regions
WC = ["Washington","Oregon","California","Nevada","Idaho"]
MW = ["Montana","Wyoming","North Dakota","South Dakota","Nebraska","Kansas","Minnesota","Iowa","Missouri",
      "Wisconsin","Illinois","Michigan","Indiana","Ohio","Kentucky"]
SW = ["Arizona","Utah","Colorado","New Mexico","Oklahoma","Texas"]
SE = ["Arkansas","Louisiana","Mississippi","Alabama","Tennessee","Virginia","West Virginia","North Carolina","South Carolina","Georgia","Florida"]
NE = ["District of Columbia","Pennsylvania","Maryland","Delaware","New Jersey","New York","Maine","New Hampshire","Vermont","Massachusetts","Rhode Island","Connecticut"]
counties["Region"]= np.where(counties["State"].isin(WC),'WC',
                    np.where(counties["State"].isin(MW),'MW',
                    np.where(counties["State"].isin(SW),'SW',
                    np.where(counties["State"].isin(SE),'SE',
                    np.where(counties["State"].isin(NE),'NE', 
                             'other')))))

In [123]:
#counties.groupby(["Region"]).sum()["TotalPop"]
cdf = pd.DataFrame(counties,columns=["county_string","Region"])
WClist = (cdf.groupby("Region").get_group("WC")["county_string"]).tolist()
MWlist = (cdf.groupby("Region").get_group("MW")["county_string"]).tolist()
SWlist = (cdf.groupby("Region").get_group("SW")["county_string"]).tolist()
SElist = (cdf.groupby("Region").get_group("SE")["county_string"]).tolist()
NElist = (cdf.groupby("Region").get_group("NE")["county_string"]).tolist()

sample_list = []
sample_list.extend(random.sample(list(set(WClist) & set(fullcountylist)),2))
sample_list.extend(random.sample(list(set(MWlist) & set(fullcountylist)),2))
sample_list.extend(random.sample(list(set(SWlist) & set(fullcountylist)),2))
sample_list.extend(random.sample(list(set(SElist) & set(fullcountylist)),2))
sample_list.extend(random.sample(list(set(NElist) & set(fullcountylist)),2))
sample_list

['Adams County, Idaho',
 'Washington County, Oregon',
 'Richardson County, Nebraska',
 'Meigs County, Ohio',
 'Chaves County, New Mexico',
 'Stonewall County, Texas',
 'Carteret County, North Carolina',
 'Cleburne County, Alabama',
 'York County, Maine',
 'Salem County, New Jersey']

In [124]:
res_df = pd.DataFrame(columns=["name","zipcode","county_string","rating","review_count","category","price"])
res_df

,name,zipcode,county_string,rating,review_count,category,price


In [128]:
r = 0
for c in sample_list:
    county_string = str(c)
    for term in ["Italian","Mexican","Chinese","American","Indian"]:
        #we are allowed 5000 queries per 24 hours. if we run out, we can create another. till then, use this
        yelpkey="IXRN_1GI8f8J9Kj5O5GciqeyFSWR5RMh41_rADGy1KrtQ5ZABGGZEkWwot5mOka0NlzlNxXPmzYXLWk88oOV0N_YxD1MSW4kQRfJnFrwfHLtKrr2HcsjLo6uWBzAWnYx"
        try:
            offset=0
            for loopcount in range(5):
                
                response = requests.get(url="https://api.yelp.com/v3/businesses/search", 
                       params= {"location": county_string,"term": term, "limit":"50", "offset": offset}, 
                       headers={'Authorization': 'bearer %s' % yelpkey})
                responsejson = response.json()
                offset += 50
                try:
                    count=0
            
                    for x in range(0,49):
                        for y in range(0,len(responsejson["businesses"][x]["categories"])):
                            
                            if ((str(responsejson["businesses"][x]["categories"][y]["title"]).startswith(str(term))) and 
                                (responsejson["businesses"][x]["review_count"] > 0)):
                        
                                zc = int(responsejson["businesses"][x]["location"]["zip_code"])
                                if (zipcodes.loc[zipcodes["zip_code"]==zc]["county_string"].tolist()[0]== county_string):
                            
#                                     print("\n",responsejson["businesses"][x]["name"],
#                                           responsejson["businesses"][x]["location"]["zip_code"],
#                                           zipcodes.loc[zipcodes["zip_code"]==zc]["county_string"].tolist()[0],
#                                           responsejson["businesses"][x]["rating"],
#                                           responsejson["businesses"][x]["review_count"],
#                                           responsejson["businesses"][x]["categories"][y]["title"])
                            
                                    res_df.loc[r]=[responsejson["businesses"][x]["name"],
                                          str(zc),
                                          county_string,
                                          responsejson["businesses"][x]["rating"],
                                          responsejson["businesses"][x]["review_count"],
                                          term,
                                          responsejson["businesses"][x]["price"]]
                                    count += 1
                                    r += 1
                except:
                    break
    
                print("End of list. Found", count, term, "restaurants in", county_string)
        except:
            break


End of list. Found 16 Italian restaurants in Washington County, Oregon
End of list. Found 9 Italian restaurants in Washington County, Oregon
End of list. Found 20 Chinese restaurants in Washington County, Oregon
End of list. Found 1 Italian restaurants in Meigs County, Ohio
End of list. Found 0 Italian restaurants in Meigs County, Ohio
End of list. Found 0 Italian restaurants in Meigs County, Ohio
End of list. Found 2 Mexican restaurants in Meigs County, Ohio
End of list. Found 0 Chinese restaurants in Meigs County, Ohio
End of list. Found 3 American restaurants in Meigs County, Ohio
End of list. Found 0 American restaurants in Meigs County, Ohio
End of list. Found 0 American restaurants in Meigs County, Ohio
End of list. Found 0 American restaurants in Meigs County, Ohio
End of list. Found 0 American restaurants in Meigs County, Ohio
End of list. Found 6 Italian restaurants in Carteret County, North Carolina
End of list. Found 0 Italian restaurants in Carteret County, North Carolina
E

In [129]:

res_df.to_csv("output1804042050.csv")
res_df.price.replace(["$","$$","$$$","$$$$"],[1,2,3,4],inplace=True)
res_df



,name,zipcode,county_string,rating,review_count,category,price
0,Li's Chinese Kitchen,83612,"Adams County, Idaho",4.5,10,Chinese,2
1,"Intersection BBQ, Grill & Bar",83654,"Adams County, Idaho",4.5,12,American,2
2,Decarli,97005,"Washington County, Oregon",4.0,534,Italian,2
3,Degarios Ristorante,97229,"Washington County, Oregon",4.0,56,Italian,2
4,Brooklyn Trattoria,97229,"Washington County, Oregon",4.0,73,Italian,2
5,Pasta Pronto,97006,"Washington County, Oregon",4.0,94,Italian,2
6,Mason - Vecchio Ristorante Mondo,97140,"Washington County, Oregon",4.5,37,Italian,2
7,Bugatti's Tanasbourne,97006,"Washington County, Oregon",3.5,173,Italian,2
8,Nonna Emilia Ristorante Italiano,97007,"Washington County, Oregon",3.0,343,Italian,2
9,Little Italy,97113,"Washington County, Oregon",3.0,21,Italian,2


In [133]:
itadf = res_df.groupby(res_df["category"]).get_group("Italian")
mexdf = res_df.groupby(res_df["category"]).get_group("Mexican")
chndf = res_df.groupby(res_df["category"]).get_group("Chinese")
usadf = res_df.groupby(res_df["category"]).get_group("American")
inddf = res_df.groupby(res_df["category"]).get_group("Indian")
res_df
# CHA = res_df.groupby(res_df["county_string"]).get_group("Mecklenburg County, North Carolina")
# CHAbreakdown = pd.DataFrame(CHA["category"].value_counts()).T.reset_index()
# CHAbreakdown["county_string"]="Mecklenburg County, North Carolina"
# CHA_price_avg = CHA.groupby(res_df["price"]).mean()
# CHA_price_count = CHA["price"].value_counts()

# RAL = res_df.groupby(res_df["county_string"]).get_group("Wake County, North Carolina")
# RALbreakdown = pd.DataFrame(RAL["category"].value_counts()).T.reset_index()
# RALbreakdown["county_string"]="Wake County, North Carolina"
# RAL_price_avg = RAL.groupby(res_df["price"]).mean()
# RAL_price_count = RAL["price"].value_counts()

# ATL = res_df.groupby(res_df["county_string"]).get_group("Fulton County, Georgia")
# ATLbreakdown = pd.DataFrame(ATL["category"].value_counts()).T.reset_index()
# ATLbreakdown["county_string"]="Fulton County, Georgia"
# ATL_price_avg = ATL.groupby(res_df["price"]).mean()
# ATL_price_count = ATL["price"].value_counts()

# MEM = res_df.groupby(res_df["county_string"]).get_group("Shelby County, Tennessee")
# MEMbreakdown = pd.DataFrame(MEM["category"].value_counts()).T.reset_index()
# MEMbreakdown["county_string"]="Shelby County, Tennessee"
# MEM_price_avg = MEM.groupby(res_df["price"]).mean()
# MEM_price_count = MEM["price"].value_counts()


# ORL = res_df.groupby(res_df["county_string"]).get_group("Orange County, Florida")
# ORLbreakdown = pd.DataFrame(ORL["category"].value_counts()).T.reset_index()
# ORLbreakdown["county_string"]="Orange County, Florida"
# ORL_price_avg = ORL.groupby(res_df["price"]).mean()
# ORL_price_count = ORL["price"].value_counts()

# MIL = res_df.groupby(res_df["county_string"]).get_group("Milwaukee County, Wisconsin")
# MILbreakdown = pd.DataFrame(MIL["category"].value_counts()).T.reset_index()
# MILbreakdown["county_string"]="Milwaukee County, Wisconsin"
# MIL_price_avg = MIL.groupby(res_df["price"]).mean()
# MIL_price_count = MIL["price"].value_counts()

# SFC = res_df.groupby(res_df["county_string"]).get_group("San Francisco County, California")
# SFCbreakdown = pd.DataFrame(SFC["category"].value_counts()).T.reset_index()
# SFCbreakdown["county_string"]="San Francisco County, California"
# SFC_price_avg = SFC.groupby(res_df["price"]).mean()
# SFC_price_count = SFC["price"].value_counts()

# AUS = res_df.groupby(res_df["county_string"]).get_group("Travis County, Texas")
# AUSbreakdown = pd.DataFrame(AUS["category"].value_counts()).T.reset_index()
# AUSbreakdown["county_string"]="Travis County, Texas"
# AUS_price_avg = AUS.groupby(res_df["price"]).mean()
# AUS_price_count = AUS["price"].value_counts()

# PIT = res_df.groupby(res_df["county_string"]).get_group("Allegheny County, Pennsylvania")
# PITbreakdown = pd.DataFrame(PIT["category"].value_counts()).T.reset_index()
# PITbreakdown["county_string"]="Allegheny County, Pennsylvania"
# PIT_price_avg = PIT.groupby(res_df["price"]).mean()
# PIT_price_count = PIT["price"].value_counts()


,name,zipcode,county_string,rating,review_count,category,price
0,Li's Chinese Kitchen,83612,"Adams County, Idaho",4.5,10,Chinese,2
1,"Intersection BBQ, Grill & Bar",83654,"Adams County, Idaho",4.5,12,American,2
2,Decarli,97005,"Washington County, Oregon",4.0,534,Italian,2
3,Degarios Ristorante,97229,"Washington County, Oregon",4.0,56,Italian,2
4,Brooklyn Trattoria,97229,"Washington County, Oregon",4.0,73,Italian,2
5,Pasta Pronto,97006,"Washington County, Oregon",4.0,94,Italian,2
6,Mason - Vecchio Ristorante Mondo,97140,"Washington County, Oregon",4.5,37,Italian,2
7,Bugatti's Tanasbourne,97006,"Washington County, Oregon",3.5,173,Italian,2
8,Nonna Emilia Ristorante Italiano,97007,"Washington County, Oregon",3.0,343,Italian,2
9,Little Italy,97113,"Washington County, Oregon",3.0,21,Italian,2


In [206]:
TOTAL_price_avg = res_df.groupby(["price"]).mean()
rescount = res_df.groupby(["county_string"]).count()["name"].reset_index()
rescount.columns=['county_string', 'restaurant_count']
TOTAL_county_reviews = res_df.groupby(["county_string"]).review_count.sum().reset_index()
TOTAL_county_reviews.columns=["county_string","review_count"]
TOTAL_county_avg = res_df.groupby(res_df["county_string"]).mean().reset_index()

county_stats = TOTAL_county_avg.merge(TOTAL_county_reviews, on="county_string")
csdf = county_stats.merge(counties, on="county_string")
c_df = rescount.merge(csdf, on="county_string")
c_df

,county_string,restaurant_count,rating,price,review_count,CensusId,State,County,TotalPop,Men,...,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,Region
0,"Adams County, Idaho",2,4.500000,2.000000,22,16003,Idaho,Adams,3880,2049,...,3.5,12.0,23.0,1474,60.4,25.2,14.2,0.1,11.3,WC
1,"Carteret County, North Carolina",35,4.057143,1.942857,1610,37031,North Carolina,Carteret,68228,33481,...,2.6,4.4,23.3,29496,66.8,22.2,10.8,0.1,10.2,SE
2,"Chaves County, New Mexico",27,3.685185,1.555556,1095,35005,New Mexico,Chaves,65811,32746,...,1.9,3.8,19.5,26448,73.6,19.1,7.2,0.1,7.6,SW
3,"Cleburne County, Alabama",1,4.500000,1.000000,15,1029,Alabama,Cleburne,15002,7334,...,1.0,3.4,33.3,6065,76.3,15.3,8.4,0.0,8.3,SE
4,"Meigs County, Ohio",6,3.666667,1.333333,53,39105,Ohio,Meigs,23473,11556,...,0.9,2.8,29.3,8770,71.9,20.4,7.5,0.2,12.1,MW
5,"Richardson County, Nebraska",1,5.000000,1.000000,16,31147,Nebraska,Richardson,8201,4081,...,0.9,7.5,18.2,4000,70.5,18.2,11.1,0.2,3.4,MW
6,"Salem County, New Jersey",9,3.611111,1.444444,188,34033,New Jersey,Salem,65120,31653,...,1.4,2.7,25.1,29226,79.7,15.6,4.6,0.1,10.6,NE
7,"Washington County, Oregon",98,3.709184,1.653061,11141,41067,Oregon,Washington,556210,273743,...,1.9,5.3,24.5,276371,84.4,9.6,5.9,0.1,7.6,WC
8,"York County, Maine",66,3.924242,2.060606,4646,23031,Maine,York,199682,97075,...,1.4,6.1,27.8,103506,78.1,14.0,7.7,0.2,5.9,NE


In [216]:
current_county="Washington County, Oregon"
WCO = res_df.groupby(res_df["county_string"]).get_group(current_county)
WCObreakdown = pd.DataFrame(WCO["category"].value_counts()).T.reset_index()
WCOrate = pd.DataFrame(WCO["rating"].value_counts()).T.reset_index()
WCObreakdown["county_string"]=current_county
WCO_price_avg = WCO.groupby(res_df["price"]).mean()
WCO_price_count = WCO["price"].value_counts()
WCOrate


,index,4.0,3.5,3.0,4.5,2.5,1.5,5.0
0,rating,42,27,14,10,3,1,1


In [ ]:
comp_df = pd.DataFrame(columns=["Italian","Mexican","Chinese","American", "Indian","county_string"])
comp_df = comp_df.append(CHAbreakdown)
comp_df = comp_df.append(RALbreakdown)
comp_df = comp_df.append(ATLbreakdown)
comp_df = comp_df.append(MEMbreakdown)
comp_df = comp_df.append(ORLbreakdown)
comp_df = comp_df.append(MILbreakdown)
comp_df = comp_df.append(SFCbreakdown)
comp_df = comp_df.append(AUSbreakdown)
comp_df = comp_df.append(PITbreakdown)
comp_df = comp_df.reset_index()
del comp_df["index"]
del comp_df["level_0"]
#comp_df["Totals"]=comp_df["Italian"]+comp_df["Mexican"]+comp_df["Chinese"]+comp_df["Indian"]+comp_df["American"]
comp_df

In [ ]:
fig, axes = plt.subplots(3,3,figsize= (16,9))
for i, (idx, row) in enumerate(comp_df.set_index("county_string").iterrows()):
    ax = axes[i // 3, i % 3]
    row = row[row.gt(row.sum()*.01)]
    ax.pie(row, labels=row.index)
    ax.set_title(idx)
fig.subplots_adjust(wspace=1)

In [ ]:
comp = pd.DataFrame(res_df["category"].groupby(res_df["county_string"]).value_counts())
comp
#I HAVE NO IDEA WHAT TO DO WITH THIS

In [ ]:
CHA = CHA["category"].value_counts().reset_index()
CHA["county_string"]="Mecklenburg County, North Carolina"
CHA.columns = ["category","count","county_string"]
CHA